# WebSocket Connection Debug - Firefox Launcher Extension

This notebook is designed to debug and fix WebSocket connection failures in the Firefox launcher extension.

## Error being debugged:
```
WebSocket connection to 'ws://192.168.7.10:8889/user/bdx/proxy/43505/' failed
```

## 1. Environment Setup and Imports

Import necessary libraries for WebSocket testing, network diagnostics, and logging.

In [3]:
import os
import sys
import socket
import time
import json
import subprocess
import logging
from typing import Optional, Dict, Any
from urllib.parse import urlparse

# Install required packages if not present
try:
    import websocket
except ImportError:
    import subprocess
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'websocket-client'])
    import websocket

try:
    import requests
except ImportError:
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'requests'])
    import requests

# Set up logging
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

print("✅ All imports successful!")
print(f"Python version: {sys.version}")
print(f"WebSocket client version: {websocket.__version__}")
print(f"Requests version: {requests.__version__}")

✅ All imports successful!
Python version: 3.12.3 (main, Feb  4 2025, 14:48:35) [GCC 13.3.0]
WebSocket client version: 1.8.0
Requests version: 2.32.4


## 2. WebSocket Connection Diagnostics

Create functions to test WebSocket connectivity, parse the failing URL, and identify specific connection issues.

In [4]:
# Failed WebSocket URL from the error log
FAILED_WS_URL = "ws://192.168.7.10:8889/user/bdx/proxy/43505/"

def parse_websocket_url(ws_url: str) -> Dict[str, Any]:
    """Parse a WebSocket URL and extract connection components."""
    parsed = urlparse(ws_url)
    return {
        'scheme': parsed.scheme,
        'host': parsed.hostname,
        'port': parsed.port,
        'path': parsed.path,
        'full_url': ws_url
    }

def test_websocket_connection(ws_url: str, timeout: int = 10) -> Dict[str, Any]:
    """Test WebSocket connection with detailed error reporting."""
    result = {
        'success': False,
        'error': None,
        'details': {},
        'url': ws_url
    }
    
    try:
        # Enable WebSocket debug logging
        websocket.enableTrace(True)
        
        # Create WebSocket connection with timeout
        ws = websocket.create_connection(
            ws_url,
            timeout=timeout,
            subprotocols=['binary']  # Xpra uses binary subprotocol
        )
        
        result['success'] = True
        result['details']['connected'] = True
        result['details']['subprotocol'] = ws.subprotocol
        
        # Close the connection
        ws.close()
        
    except websocket.WebSocketException as e:
        result['error'] = f"WebSocket error: {e}"
        result['details']['websocket_error'] = str(e)
    except socket.timeout as e:
        result['error'] = f"Connection timeout: {e}"
        result['details']['timeout'] = True
    except socket.error as e:
        result['error'] = f"Socket error: {e}"
        result['details']['socket_error'] = str(e)
    except Exception as e:
        result['error'] = f"General error: {e}"
        result['details']['general_error'] = str(e)
    
    return result

# Parse the failed URL
parsed_url = parse_websocket_url(FAILED_WS_URL)
print("📋 Parsed WebSocket URL:")
for key, value in parsed_url.items():
    print(f"  {key}: {value}")

# Test the connection
print("\n🔍 Testing WebSocket connection...")
connection_result = test_websocket_connection(FAILED_WS_URL)
print(f"Connection result: {json.dumps(connection_result, indent=2)}")

--- request header ---
2025-08-05 02:59:13,444 - DEBUG - --- request header ---
GET /user/bdx/proxy/43505/ HTTP/1.1
Upgrade: websocket
Host: 192.168.7.10:8889
Origin: http://192.168.7.10:8889
Sec-WebSocket-Key: jqN+erU7A9fK1frIAg5cOQ==
Sec-WebSocket-Version: 13
Connection: Upgrade
Sec-WebSocket-Protocol: binary


2025-08-05 02:59:13,445 - DEBUG - GET /user/bdx/proxy/43505/ HTTP/1.1
Upgrade: websocket
Host: 192.168.7.10:8889
Origin: http://192.168.7.10:8889
Sec-WebSocket-Key: jqN+erU7A9fK1frIAg5cOQ==
Sec-WebSocket-Version: 13
Connection: Upgrade
Sec-WebSocket-Protocol: binary


-----------------------
2025-08-05 02:59:13,447 - DEBUG - -----------------------
--- response header ---
2025-08-05 02:59:13,447 - DEBUG - --- response header ---
HTTP/1.1 403 Forbidden
2025-08-05 02:59:13,449 - DEBUG - HTTP/1.1 403 Forbidden
Server: TornadoServer/6.5.1
2025-08-05 02:59:13,451 - DEBUG - Server: TornadoServer/6.5.1
Content-Type: text/html; charset=UTF-8
2025-08-05 02:59:13,452 - DEBUG - Content-T

📋 Parsed WebSocket URL:
  scheme: ws
  host: 192.168.7.10
  port: 8889
  path: /user/bdx/proxy/43505/
  full_url: ws://192.168.7.10:8889/user/bdx/proxy/43505/

🔍 Testing WebSocket connection...
Connection result: {
  "success": false,
  "error": "WebSocket error: Handshake status 403 Forbidden -+-+- {'server': 'TornadoServer/6.5.1', 'content-type': 'text/html; charset=UTF-8', 'date': 'Tue, 05 Aug 2025 02:59:13 GMT', 'content-length': '9'} -+-+- b'Forbidden'",
  "details": {
    "websocket_error": "Handshake status 403 Forbidden -+-+- {'server': 'TornadoServer/6.5.1', 'content-type': 'text/html; charset=UTF-8', 'date': 'Tue, 05 Aug 2025 02:59:13 GMT', 'content-length': '9'} -+-+- b'Forbidden'"
  },
  "url": "ws://192.168.7.10:8889/user/bdx/proxy/43505/"
}


## 3. Network Connectivity Testing

Implement network tests to check if the target host is reachable using ping, telnet, and socket connections.

In [ ]:
def test_host_reachability(host: str) -> Dict[str, Any]:
    """Test if a host is reachable via ping."""
    result = {'host': host, 'reachable': False, 'error': None}
    
    try:
        # Use ping command to test reachability
        cmd = ['ping', '-c', '1', '-W', '3', host]
        process = subprocess.run(cmd, capture_output=True, text=True, timeout=10)
        
        if process.returncode == 0:
            result['reachable'] = True
            result['ping_output'] = process.stdout.strip()
        else:
            result['error'] = f"Ping failed: {process.stderr.strip()}"
            
    except subprocess.TimeoutExpired:
        result['error'] = "Ping timeout"
    except Exception as e:
        result['error'] = f"Ping error: {e}"
    
    return result

def test_port_connectivity(host: str, port: int, timeout: int = 5) -> Dict[str, Any]:
    """Test if a specific port is open on a host."""
    result = {'host': host, 'port': port, 'open': False, 'error': None}
    
    try:
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        sock.settimeout(timeout)
        
        result_code = sock.connect_ex((host, port))
        sock.close()
        
        if result_code == 0:
            result['open'] = True
        else:
            result['error'] = f"Port {port} closed or filtered"
            
    except socket.gaierror as e:
        result['error'] = f"DNS resolution failed: {e}"
    except Exception as e:
        result['error'] = f"Socket error: {e}"
    
    return result

def test_http_connectivity(host: str, port: int, path: str = "/") -> Dict[str, Any]:
    """Test HTTP connectivity to a host:port combination."""
    result = {'host': host, 'port': port, 'path': path, 'success': False, 'error': None}
    
    try:
        url = f"http://{host}:{port}{path}"
        response = requests.get(url, timeout=10)
        
        result['success'] = True
        result['status_code'] = response.status_code
        result['headers'] = dict(response.headers)
        result['url'] = url
        
    except requests.exceptions.RequestException as e:
        result['error'] = f"HTTP request failed: {e}"
    except Exception as e:
        result['error'] = f"General error: {e}"
    
    return result

# Test network connectivity to the target host
target_host = parsed_url['host']
target_port = parsed_url['port']

print("🌐 Testing network connectivity...")

# Test host reachability
ping_result = test_host_reachability(target_host)
print(f"\n📡 Ping test to {target_host}:")
print(f"  Reachable: {ping_result['reachable']}")
if ping_result['error']:
    print(f"  Error: {ping_result['error']}")

# Test port connectivity
port_result = test_port_connectivity(target_host, target_port)
print(f"\n🔌 Port test to {target_host}:{target_port}:")
print(f"  Open: {port_result['open']}")
if port_result['error']:
    print(f"  Error: {port_result['error']}")

# Test HTTP connectivity to proxy path
http_result = test_http_connectivity(target_host, target_port, parsed_url['path'])
print(f"\n🌍 HTTP test to {target_host}:{target_port}{parsed_url['path']}:")
print(f"  Success: {http_result['success']}")
if http_result['error']:
    print(f"  Error: {http_result['error']}")
elif http_result['success']:
    print(f"  Status: {http_result['status_code']}")
    print(f"  Server: {http_result['headers'].get('Server', 'Unknown')}")

## 4. Port and Proxy Configuration Check

Verify if port 8889 is open and accessible, check proxy settings, and test direct vs proxied connections.

In [5]:
def check_proxy_routes() -> Dict[str, Any]:
    """Check the current proxy routes in JupyterHub."""
    result = {'success': False, 'routes': {}, 'error': None}
    
    try:
        # Try to get proxy routes (without auth first)
        proxy_api_url = "http://127.0.0.1:8001/api/routes"
        response = requests.get(proxy_api_url, timeout=5)
        
        if response.status_code == 200:
            result['success'] = True
            result['routes'] = response.json()
        else:
            result['error'] = f"HTTP {response.status_code}: {response.text}"
            
    except Exception as e:
        result['error'] = f"Request failed: {e}"
    
    return result

def check_xpra_direct_connection(port: int) -> Dict[str, Any]:
    """Test direct connection to Xpra server."""
    result = {'port': port, 'success': False, 'error': None}
    
    # Test different hosts
    hosts_to_try = ['localhost', '127.0.0.1', '192.168.7.10']
    
    for host in hosts_to_try:
        try:
            url = f"http://{host}:{port}/"
            response = requests.get(url, timeout=5)
            
            if response.status_code == 200:
                result['success'] = True
                result['working_host'] = host
                result['status_code'] = response.status_code
                result['server'] = response.headers.get('Server', 'Unknown')
                break
                
        except Exception as e:
            result[f'error_{host}'] = str(e)
    
    if not result['success']:
        result['error'] = "Could not connect to Xpra server on any host"
    
    return result

def get_environment_info() -> Dict[str, Any]:
    """Get environment information related to proxy configuration."""
    env_info = {}
    
    # Check for JupyterHub proxy environment variables
    proxy_vars = ['CONFIGPROXY_API_URL', 'CONFIGPROXY_AUTH_TOKEN', 'JUPYTERHUB_API_TOKEN']
    for var in proxy_vars:
        value = os.environ.get(var)
        env_info[var] = '[SET]' if value else '[NOT SET]'
    
    # Check current working directory and user
    env_info['PWD'] = os.getcwd()
    env_info['USER'] = os.environ.get('USER', 'unknown')
    env_info['JUPYTER_SERVER_ROOT'] = os.environ.get('JUPYTER_SERVER_ROOT', '[NOT SET]')
    
    return env_info

# Check proxy routes
print("🔍 Checking JupyterHub proxy routes...")
proxy_routes = check_proxy_routes()
if proxy_routes['success']:
    print("✅ Successfully retrieved proxy routes:")
    for route, target in proxy_routes['routes'].items():
        print(f"  {route} -> {target}")
        
    # Check if our specific route exists
    target_route = f"/user/bdx/proxy/43505/"
    if target_route in proxy_routes['routes']:
        print(f"\n✅ Found our target route: {target_route} -> {proxy_routes['routes'][target_route]}")
    else:
        print(f"\n❌ Our target route {target_route} is NOT registered!")
        print("Available routes:")
        for route in proxy_routes['routes'].keys():
            if 'proxy' in route:
                print(f"  {route}")
else:
    print(f"❌ Failed to get proxy routes: {proxy_routes['error']}")

# Test direct Xpra connection
print(f"\n🔧 Testing direct connection to Xpra server on port 43505...")
xpra_result = check_xpra_direct_connection(43505)
if xpra_result['success']:
    print(f"✅ Xpra server is accessible at {xpra_result['working_host']}:43505")
    print(f"  Server: {xpra_result['server']}")
else:
    print(f"❌ Could not connect to Xpra server: {xpra_result['error']}")

# Show environment info
print(f"\n📋 Environment information:")
env_info = get_environment_info()
for key, value in env_info.items():
    print(f"  {key}: {value}")

2025-08-05 02:59:22,029 - DEBUG - Starting new HTTP connection (1): 127.0.0.1:8001
2025-08-05 02:59:22,031 - DEBUG - http://127.0.0.1:8001 "GET /api/routes HTTP/1.1" 403 69
2025-08-05 02:59:22,032 - DEBUG - Starting new HTTP connection (1): localhost:43505
2025-08-05 02:59:22,039 - DEBUG - http://localhost:43505 "GET / HTTP/1.1" 200 13281


🔍 Checking JupyterHub proxy routes...
❌ Failed to get proxy routes: HTTP 403: <html><title>403: Forbidden</title><body>403: Forbidden</body></html>

🔧 Testing direct connection to Xpra server on port 43505...
✅ Xpra server is accessible at localhost:43505
  Server: Xpra-WebSocket-Server Python/3.12.3

📋 Environment information:
  CONFIGPROXY_API_URL: [NOT SET]
  CONFIGPROXY_AUTH_TOKEN: [NOT SET]
  JUPYTERHUB_API_TOKEN: [NOT SET]
  PWD: /home/bdx/allcode/github/vantagecompute/jup-fir-lau
  USER: bdx
  JUPYTER_SERVER_ROOT: [NOT SET]
